In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autoreload
%autoreload 2

## Check clip sizes

In [2]:
from Cfg import Cfg
C = Cfg('NIST', 16000, 'amharic', 'build') 

splits=C.split_files()

L=[x['t_seconds'] for x in splits]

In [3]:
min(L),max(L),sum(L)/len(L)

(0.135, 32.51, 3.6957927164784175)

## Train step 1: Bootstrap from pretrained model

In [4]:
from Cfg import Cfg
C = Cfg('NIST', 16000, 'amharic', 'build') 

In [5]:
from load_pretrained_amharic_model import load_pretrained_amharic_model
model = load_pretrained_amharic_model(C, 0)

[NeMo W 2020-10-17 00:19:48 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-17 00:19:48 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToBPEDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-17 00:19:48 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioLabelDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-17 00:19:48 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text._TarredAudioToTextDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-17 00:19:48 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.TarredAudioToCharDataset'> is experimental,

[NeMo I 2020-10-17 00:19:49 features:241] PADDING: 16
[NeMo I 2020-10-17 00:19:49 features:258] STFT using torch
loaded save/nemo_amharic/amharic_20201016_191843_62677_epoch=75.ckpt


In [6]:
from reshuffle_samples import reshuffle_samples
reshuffle_samples(C)

In [7]:
import pytorch_lightning as pl
import os, datetime

model_save_dir='save/nemo_amharic'

class ModelCheckpointAtEpochEnd(pl.callbacks.ModelCheckpoint):
    def on_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        metrics['epoch'] = trainer.current_epoch
        trainer.checkpoint_callback.on_validation_end(trainer, pl_module)

pid=os.getpid()
dt=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

checkpoint_callback = ModelCheckpointAtEpochEnd(
    filepath=model_save_dir+'/amharic_'+f'{dt}_{pid}'+'_{epoch:02d}',
    verbose=True,
    save_top_k=-1,
    save_weights_only=False,
    period=1)

trainer = pl.Trainer(gpus=[0], max_epochs=1000, amp_level='O1', precision=16, checkpoint_callback=checkpoint_callback)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


In [8]:
from ruamel.yaml import YAML
from omegaconf import DictConfig
config_path = 'amharic_16000.yaml'
yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
train_manifest=f'{C.build_dir}/train_manifest.json'
test_manifest=f'{C.build_dir}/test_manifest.json'
params['model']['train_ds']['manifest_filepath'] = train_manifest
params['model']['validation_ds']['manifest_filepath'] = test_manifest
model.set_trainer(trainer)
model.setup_training_data(train_data_config=params['model']['train_ds'])
model.setup_validation_data(val_data_config=params['model']['validation_ds'])
model.setup_optimization(optim_config=DictConfig(params['model']['optim']))

[NeMo I 2020-10-17 00:19:51 collections:173] Dataset loaded with 9323 files totalling 9.57 hours
[NeMo I 2020-10-17 00:19:51 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2020-10-17 00:19:51 collections:173] Dataset loaded with 1036 files totalling 1.06 hours
[NeMo I 2020-10-17 00:19:51 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2020-10-17 00:19:51 modelPT:572] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 5e-05
        weight_decay: 0.001
    )
[NeMo I 2020-10-17 00:19:51 lr_scheduler:545] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f825d6ded90>" 
    will be used during training (effective maximum steps = 776917) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 776917
    )


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.8, 0.5]
     eps: 1e-08
     grad_averaging: False
     initial_lr: 5e-05
     lr: 5e-05
     weight_decay: 0.001
 ),
 {'scheduler': <nemo.core.optim.lr_scheduler.CosineAnnealing at 0x7f825d6ded90>,
  'interval': 'step',
  'frequency': 1,
  'monitor': 'val_loss',
  'reduce_on_plateau': False})

In [9]:
trainer.fit(model)

[NeMo I 2020-10-17 00:19:51 modelPT:572] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 5e-05
        weight_decay: 0.001
    )
[NeMo I 2020-10-17 00:19:51 lr_scheduler:545] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f825d295ad0>" 
    will be used during training (effective maximum steps = 776917) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 776917
    )


[NeMo W 2020-10-17 00:19:52 nemo_logging:349] /home/catskills/anaconda3/envs/nemo/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning:
    
    Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
    
    

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18 M  
2 | decoder           | ConvASRDecoder                    | 241 K 
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     


[NeMo I 2020-10-17 00:19:53 wer:148] 
    
[NeMo I 2020-10-17 00:19:53 wer:149] reference:ጥያቄ አላሰብኩበትም እንጂ ጥያቄዎች ብዙ አሉ
[NeMo I 2020-10-17 00:19:53 wer:150] decoded  :ጥያቄ አላሰብኩበትም እንጂ ጥያቄዎች ብዙ አሉ


[NeMo W 2020-10-17 00:19:54 nemo_logging:349] /home/catskills/anaconda3/envs/nemo/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning:
    
    Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
    
    


[NeMo I 2020-10-17 00:20:07 wer:148] 
    
[NeMo I 2020-10-17 00:20:07 wer:149] reference:እሺ እመጣለሁ ሐሙስ ነው የምጨርሱ ፈተና
[NeMo I 2020-10-17 00:20:07 wer:150] decoded  :እሺ እመጣለሁ ሐሙስ ነው የምጨርሱ ፈተና
[NeMo I 2020-10-17 00:20:22 wer:148] 
    
[NeMo I 2020-10-17 00:20:22 wer:149] reference:ደህና ነው ደህና ነው እግዚአብሔር ይመስገን ደህና ነው አባዬስ ደህና ነች
[NeMo I 2020-10-17 00:20:22 wer:150] decoded  :ደና ነው ደህና ነው እግዚአብሔር ይመንን ደህና ነው አባዬስ ደህና ነች
[NeMo I 2020-10-17 00:20:36 wer:148] 
    
[NeMo I 2020-10-17 00:20:36 wer:149] reference:አሁን ሥራ እየሠራ ነው
[NeMo I 2020-10-17 00:20:36 wer:150] decoded  :አሁን ሥራ እየሠራ ነው
[NeMo I 2020-10-17 00:20:51 wer:148] 
    
[NeMo I 2020-10-17 00:20:51 wer:149] reference:አለን
[NeMo I 2020-10-17 00:20:51 wer:150] decoded  :ኧረ
[NeMo I 2020-10-17 00:21:06 wer:148] 
    
[NeMo I 2020-10-17 00:21:06 wer:149] reference:ምን ብገዛ ይሻላል
[NeMo I 2020-10-17 00:21:06 wer:150] decoded  :ምን ብገዛ ይሻላል
[NeMo I 2020-10-17 00:21:20 wer:148] 
    
[NeMo I 2020-10-17 00:21:20 wer:149] reference:አዎ አይ ኤደንዬ እኔ እኮ የገረ

...

[NeMo I 2020-10-17 00:39:29 wer:148] 
    
[NeMo I 2020-10-17 00:39:29 wer:149] reference:ብዙ ከመፈረሜ የተነሳ የእጅ ጽሁፌ ሁላ ተሻሻለ እኮ
[NeMo I 2020-10-17 00:39:29 wer:150] decoded  :ብዙ ከመፈረሜ የተነሳ የእጅ ጽሁ ቅ መ ተሻሻለ እኮ
[NeMo I 2020-10-17 00:39:44 wer:148] 
    
[NeMo I 2020-10-17 00:39:44 wer:149] reference:የኢትዮጵያ ከለራችን አንዱ ይሄ ነው
[NeMo I 2020-10-17 00:39:44 wer:150] decoded  :የኢትዮጵያ ከለችን አንዱ ይ ነው
[NeMo I 2020-10-17 00:39:58 wer:148] 
    
[NeMo I 2020-10-17 00:39:58 wer:149] reference:ሄሎ አቤት ሸዊት ሰላም ነው
[NeMo I 2020-10-17 00:39:58 wer:150] decoded  :ሄሎ አቤት ሸዊት ሰላም ነው
[NeMo I 2020-10-17 00:40:12 wer:148] 
    
[NeMo I 2020-10-17 00:40:12 wer:149] reference:አሁን ወደ እዛ ልትሄጂ ነዉማለት ነው
[NeMo I 2020-10-17 00:40:12 wer:150] decoded  :አሁን ወደ እዛ ልትሄጂ ነዉማለት ነው
[NeMo I 2020-10-17 00:40:26 wer:148] 
    
[NeMo I 2020-10-17 00:40:26 wer:149] reference:አሁን መቼ ዕለት
[NeMo I 2020-10-17 00:40:26 wer:150] decoded  :አሁን መቼ ዕለት
[NeMo I 2020-10-17 00:40:40 wer:148] 
    
[NeMo I 2020-10-17 00:40:40 wer:149] reference:ለምን
[NeMo

[NeMo I 2020-10-17 00:42:59 wer:148] 
    
[NeMo I 2020-10-17 00:42:59 wer:149] reference:ጥያቄ አላሰብኩበትም እንጂ ጥያቄዎች ብዙ አሉ
[NeMo I 2020-10-17 00:42:59 wer:150] decoded  :ጥያቄ አላሰብኩበትም እንጂ ጥያቄዎች ብዙ አሉ
[NeMo I 2020-10-17 00:42:59 wer:148] 
    
[NeMo I 2020-10-17 00:42:59 wer:149] reference:የ ይላል
[NeMo I 2020-10-17 00:42:59 wer:150] decoded  :የ ይላል
[NeMo I 2020-10-17 00:42:59 wer:148] 
    
[NeMo I 2020-10-17 00:42:59 wer:149] reference:አዎ እኮ እሱ እሱ አለ እሱ እንዳለ ሆኖ ነው እኮ ነገም ደግሞ ሥራ የማገኝ ልጅ ነኝ
[NeMo I 2020-10-17 00:42:59 wer:150] decoded  :አዎ እኮ እሱ እሱ አለ እሱ እንዳለ ሆኖ ነው እኮ ነገም ደግሞ ሥራ የማገኝ ልጅ ነኝ
[NeMo I 2020-10-17 00:42:59 wer:148] 
    
[NeMo I 2020-10-17 00:42:59 wer:149] reference:አሁን የካ ፓርክ ነን አዎ እየተዝናናሁ ነው
[NeMo I 2020-10-17 00:42:59 wer:150] decoded  :አሁን የካ ፓርክ ነን አዎ እየተዝናናሁ ነው
[NeMo I 2020-10-17 00:42:59 wer:148] 
    
[NeMo I 2020-10-17 00:42:59 wer:149] reference:አዎ ደርሷል አንድ አራት ቀን ነው የቀረው እኔ እንኳን አልፆምኩም አዎ እኔ አልፆምኩም አልፎኛል በአሁኑ
[NeMo I 2020-10-17 00:42:59 wer:150] decoded  :አዎ ደርሷል አንድ አራት

[NeMo I 2020-10-17 00:43:02 wer:148] 
    
[NeMo I 2020-10-17 00:43:02 wer:149] reference:ውድ ነው በይኛ ስንት ተከራየሽ
[NeMo I 2020-10-17 00:43:02 wer:150] decoded  :ውድ ነው በይኛ ስንት ተከራየሽ
[NeMo I 2020-10-17 00:43:02 wer:148] 
    
[NeMo I 2020-10-17 00:43:02 wer:149] reference:ደህና ናት በጣም ደህና ናት እግዚአብሔር ይመስገን አንቺስ ቤተሰብ እንደት ናቸው
[NeMo I 2020-10-17 00:43:02 wer:150] decoded  :ደህና ናት በጣም ደህና ናት እግዚአብሔር ይመስገን አንቺስ ቤተሰብ እንደት ናቸው
[NeMo I 2020-10-17 00:43:02 wer:148] 
    
[NeMo I 2020-10-17 00:43:02 wer:149] reference:ኦኬ ፖስት ግራጁዌት ነህ እንደ
[NeMo I 2020-10-17 00:43:02 wer:150] decoded  :ኦኬ ፖስት ግራጁዌት ነህ እንደ
[NeMo I 2020-10-17 00:43:02 wer:148] 
    
[NeMo I 2020-10-17 00:43:02 wer:149] reference:እማዬ አበባው ሁሉም ሰላም ናቸው
[NeMo I 2020-10-17 00:43:02 wer:150] decoded  :እማዬ አበባው ሁሉም ሰላም ናቸው
[NeMo I 2020-10-17 00:43:02 wer:148] 
    
[NeMo I 2020-10-17 00:43:02 wer:149] reference:አዎ አዎ ሳሪስ ነኝ አልሽኝ
[NeMo I 2020-10-17 00:43:02 wer:150] decoded  :አዎ አዎ ሳሪስ ነኝ አልሽኝ
[NeMo I 2020-10-17 00:43:02 wer:148] 
    
[NeMo I 2020


Epoch 00005: saving model to /home/catskills/Desktop/openasr20/save/nemo_amharic/amharic_20201017_001951_101244_epoch=05.ckpt
[NeMo W 2020-10-17 00:43:15 nemo_logging:349] /home/catskills/anaconda3/envs/nemo/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning:
    
    Detected KeyboardInterrupt, attempting graceful shutdown...
    
    
Saving latest checkpoint..


1

## DEV translation

In [ ]:
from Cfg import Cfg
from glob import glob
from package_DEV import package_DEV
from load_pretrained_amharic_model import load_pretrained_amharic_model
version='113'
C = Cfg('NIST', 16000, 'amharic', 'dev', version)
model = load_pretrained_amharic_model(C, 0)
files=list(sorted(glob(f'{C.audio_split_dir}/*.wav')))
translations=model.transcribe(paths2audio_files=files, batch_size=32)